In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc
import glob

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Projection_est_in_Q3'

In [2]:
sales_file_2018Q3=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
sales_file_2018Q3=[x for x in sales_file_2018Q3 if x.split("MediaStormDailySalesHistory")[1][:8]>"20180804" and x.split("MediaStormDailySalesHistory")[1][:8]<="20181103"]
len(sales_file_2018Q3)

13

In [3]:
df_product_taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190901-135942-034.txt",
                                 sep="|",dtype=str)
df_product_taxonomy=df_product_taxonomy[['division_id','department_id','class_code_id']].drop_duplicates()
df_product_taxonomy=df_product_taxonomy[df_product_taxonomy['division_id']=="10"]
df_product_taxonomy.shape

(23, 3)

In [10]:
# Division 10 only

df_agg_POS_data_2018Q3_rewards=pd.DataFrame()
df_agg_POS_data_2018Q3_nonrewards=pd.DataFrame()

i_counter=0
for file in sales_file_2018Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed',
                                                    'class_code_id','subclass_id','item_transaction_units','item_transaction_amt'])
    df=pd.merge(df,df_product_taxonomy,on="class_code_id",how="inner")
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['customer_id_hashed']=df['customer_id_hashed'].fillna("non_rewarsd")
    
    df=df.groupby(['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    
    df_r=df[df['customer_id_hashed']!="non_rewarsd"]
    df_non_r=df[df['customer_id_hashed']=="non_rewarsd"]

    df_agg_POS_data_2018Q3_rewards=df_agg_POS_data_2018Q3_rewards.append(df_r)
    df_agg_POS_data_2018Q3_nonrewards=df_agg_POS_data_2018Q3_nonrewards.append(df_non_r)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2018Q3_rewards.shape,df_agg_POS_data_2018Q3_nonrewards.shape)
    i_counter+=1
del df_agg_POS_data_2018Q3_nonrewards['customer_id_hashed']
gc.collect()

2019-09-16 15:08:59.612785 0 (60715, 8) (73505, 8)
2019-09-16 15:11:38.695804 1 (122957, 8) (145461, 8)
2019-09-16 15:14:15.508870 2 (185229, 8) (216911, 8)
2019-09-16 15:16:50.128704 3 (247729, 8) (288383, 8)
2019-09-16 15:19:20.215272 4 (312451, 8) (362393, 8)
2019-09-16 15:21:43.359832 5 (374014, 8) (432231, 8)
2019-09-16 15:23:54.706508 6 (436084, 8) (503230, 8)
2019-09-16 15:27:15.398823 7 (513135, 8) (565720, 8)
2019-09-16 15:30:37.805232 8 (588051, 8) (649721, 8)
2019-09-16 15:32:56.042342 9 (652217, 8) (726850, 8)
2019-09-16 15:35:16.229397 10 (716451, 8) (802980, 8)
2019-09-16 15:37:45.852889 11 (780812, 8) (878659, 8)
2019-09-16 15:40:51.751743 12 (846309, 8) (955394, 8)


105

In [45]:
df_agg_POS_data_2018Q3_rewards['item_transaction_amt'].sum(),df_agg_POS_data_2018Q3_nonrewards['item_transaction_amt'].sum()

(2291230.5100000002, 2137183.9099999997)

In [53]:
df_agg_rewards=df_agg_POS_data_2018Q3_rewards.groupby(['class_code_id','subclass_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index()
df_agg_nonrewards=df_agg_POS_data_2018Q3_nonrewards.groupby(['class_code_id','subclass_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index()


In [54]:
df_agg_rewards['rewards_level']="rewards"
df_agg_nonrewards['rewards_level']="nonrewards"

In [55]:
df_total_18Q3_div10=df_agg_rewards.append(df_agg_nonrewards)
df_total_18Q3_div10=pd.merge(df_total_18Q3_div10,df_product_taxonomy,on="class_code_id",how="left")


In [56]:
div_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str).drop_duplicates()
dept_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str).drop_duplicates()
class_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding="ISO-8859-1").drop_duplicates()
subclass_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Subclass Names.txt",sep="|",dtype=str,encoding="ISO-8859-1").drop_duplicates()


In [57]:
df_total_18Q3_div10=pd.merge(df_total_18Q3_div10,div_name,on="division_id",how="left")
df_total_18Q3_div10=pd.merge(df_total_18Q3_div10,dept_name,on="department_id",how="left")
df_total_18Q3_div10=pd.merge(df_total_18Q3_div10,class_name,on="class_code_id",how="left")
df_total_18Q3_div10=pd.merge(df_total_18Q3_div10,subclass_name,on=["class_code_id","subclass_id"],how="left")
df_total_18Q3_div10.shape

(2919, 11)

In [58]:
df_total_18Q3_div10['item_transaction_amt'].sum()

4428414.4200000046

In [64]:
df_5_weeks_only=df_total_18Q3_div10[df_total_18Q3_div10['transaction_dt']<="2018-09-08"]

In [65]:
writer=pd.ExcelWriter("./QC_2018Division_10_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_5_weeks_only.to_excel(writer,"df_5_weeks_only",index=False)
df_total_18Q3_div10.to_excel(writer,"df_13_weeks_all",index=False)
writer.save()